Camel 3 Function 
=====================


In [2]:
include("notebook_setup.jl")

🚀 Setting up GlobtimPlots notebook environment
📦 Activating Globtim environment at: /Users/ghscholt/globtim


  Activating 

📦 Adding GlobtimPlots in dev mode from: /Users/ghscholt/globtimplots/

project at `~/globtim`


   Resolving package versions...



📋 Loading required packages...
  ✅ Globtim
  ✅ GlobtimPlots
  ✅ CairoMakie
  ✅ DataFrames

✅ Notebook setup complete!
📋 Available from Globtim:
  - camel function
  - test_input constructor
  - Constructor for polynomial approximation
  - RationalPrecision
  - solve_polynomial_system
  - process_crit_pts
  - analyze_critical_points
📋 Available from GlobtimPlots:
  - cairo_plot_polyapprox_levelset
  - AbstractPolynomialData, AbstractProblemInput
  - adapt_polynomial_data, adapt_problem_input


  No Changes to `~/globtim/Project.toml`
  No Changes to `~/globtim/Manifest.toml`


We define the center and range of the square domain of approximation. 

In [3]:
# Constants and Parameters
const n, a, b = 2, 5, 1
const scale_factor = a / b
f = camel # Objective function
d = 6 # Initial Degree 
SMPL = 200   
println("Number of samples: ", SMPL^2)
TR = test_input(f, 
                dim = n,
                center = [0.0, 0.0],
                GN = SMPL, 
                sample_range = scale_factor
                )

Number of samples: 40000


test_input(2, [0.0, 0.0], 200, (0.1, 0.5), 0.002, (0.0, 0.0), 5.0, 1.0, 6, Globtim.camel)

The `Constructor` function computes the coefficients of the polynomial approximant in the basis specified, with `GN^2` samples distributed following the measure with respect to which the polynomial basis is orthogonal.
The `RationalPrecision` specifies that the conversion of the polynomial's coefficients from the tensorized orthogonal basis to the standard monomial basis is carried out in high precision. 

In [4]:
pol_cheb = Constructor(TR, d, basis=:chebyshev, precision=RationalPrecision)
pol_lege = Constructor(TR, d, basis=:legendre, precision=RationalPrecision, normalized=true);

current L2-norm: 5.4524172392659695e-12
current L2-norm: 4.6461539462782044e-12


Note that with the `Legendre` polynomials, it is necessary to run with the `normalized` option to ensure that the polynomials are orthonormal with respect to the uniform measure on the domain of approximation.

In [5]:
@polyvar(x[1:n]) # Define polynomial ring 
real_pts_cheb = solve_polynomial_system(
    x, n, d, pol_cheb.coeffs;
    basis=pol_cheb.basis,
    precision=pol_cheb.precision,
    normalized=pol_cheb.normalized,
)

real_pts_lege = solve_polynomial_system(
    x, n, d, pol_lege.coeffs;
    basis=pol_lege.basis,
    precision=pol_lege.precision,
    normalized=pol_lege.normalized)

df_cheb = process_crit_pts(real_pts_cheb, f, TR)
df_lege = process_crit_pts(real_pts_lege, f, TR);

Tracking 25 paths...   8%|██▍                           |  ETA: 0:01:04



Tracking 25 paths... 100%|██████████████████████████████| Time: 0:00:05
                   # paths tracked: 25
   # non-singular solutions (real): 25 (17)
       # singular endpoints (real): 0 (0)
          # total solutions (real): 25 (17)
Tracking 25 paths...   8%|██▍                           |  ETA: 0:00:15



Tracking 25 paths... 100%|██████████████████████████████| Time: 0:00:01
                   # paths tracked: 25
   # non-singular solutions (real): 25 (17)
       # singular endpoints (real): 0 (0)
          # total solutions (real): 25 (17)


In [6]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.00125);
df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR, tol_dist=0.00125);

Processing point 1 of 15
Optimization has converged within bounds: ✓
Processing point 2 of 15
Optimization has converged within bounds: ✓
Processing point 3 of 15
Optimization has converged within bounds: ✓
Processing point 4 of 15
Optimization has converged within bounds: ✓
Processing point 5 of 15
Optimization has converged within bounds: ✓
Processing point 6 of 15
Optimization has converged within bounds: ✓
Processing point 7 of 15
Optimization has converged within bounds: ✓
Processing point 8 of 15
Optimization has converged within bounds: ✓
Processing point 9 of 15
Optimization has converged within bounds: ✓
Processing point 10 of 15
Optimization has converged within bounds: ✓
Processing point 11 of 15
Optimization has converged within bounds: ✓
Processing point 12 of 15
Optimization has converged within bounds: ✓
Processing point 13 of 15
Optimization has converged within bounds: ✓
Processing point 14 of 15
Optimization has converged within bounds: ✓
Processing point 15 of 15
Opt

In [7]:
adapted_poly_cheb = adapt_polynomial_data(pol_cheb)
adapted_poly_lege = adapt_polynomial_data(pol_lege)
adapted_input = adapt_problem_input(TR)

GenericProblemInput(2, [0.0, 0.0], 5.0)

In [8]:
using GLMakie

┌ Warning: Module GlobtimPlots with build ID fafbfcfd-f2b5-40b9-2cfa-a36b59199162 is missing from the cache.
│ This may mean GlobtimPlots [12345678-1234-1234-1234-123456789abc] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:2541
┌ Error: Error during loading of extension GlobtimGLMakieExt of GlobtimPlots, use `Base.retry_load_extensions()` to retry.
│   exception = 1-element ExceptionStack:
LoadError: ArgumentError: Package GlobtimGLMakieExt does not have DynamicPolynomials in its dependencies:
- You may have a partially installed environment. Try `Pkg.instantiate()`
  to ensure all packages in the environment are installed.
- Or, if you have GlobtimGLMakieExt checked out for development and have
  added DynamicPolynomials as a dependency but haven't updated your primary
  environment's manifest file, try `Pkg.resolve()`.
- Otherwise you may need to report an issue with GlobtimGLMakieExt
Stacktrace:
  [1] macro expansion
    @ ./loading.jl:22

In [ ]:
fig_3d = plot_polyapprox_3d(
    adapted_poly_cheb,
    adapted_input,
    df_cheb,
    df_min_cheb;
    rotate=true,  # Optional animation
    fade=true     # Optional fade effect
)